# Sentiment Analysis with LSTM-GRU

In [1]:
import warnings
warnings.filterwarnings("ignore")
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf

**IMDB Dataset**

In [3]:
df = pd.read_csv("./IMDB Dataset.csv")

In [4]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
df = df.sample(10000,random_state = 1)

In [6]:
df.sentiment.value_counts()

negative    5044
positive    4956
Name: sentiment, dtype: int64

**Preprocessing and Text to Sequence**

In [7]:
from keras_preprocessing.text import Tokenizer
num_words = 10000
tokenizer = Tokenizer(num_words=10000,oov_token="<OOV>")



In [8]:
X = df.review.values
y = df.sentiment.values

In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test = train_test_split(X,y, test_size=.2, random_state=1)

In [11]:
tokenizer.fit_on_texts(X_train)

In [12]:
train_sequences= tokenizer.texts_to_sequences(X_train)
test_sequences= tokenizer.texts_to_sequences(X_test)

In [13]:
vocabulary_size = len(tokenizer.word_index)+1

In [14]:
maxlen = len(max(train_sequences,key=len))
maxlen

1853

In [15]:
from keras_preprocessing.sequence import pad_sequences
padded_train_sequences = pad_sequences(train_sequences,maxlen=maxlen)
padded_test_sequences = pad_sequences(test_sequences,maxlen=maxlen)

**LSTM Model**

In [22]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

model_LSTM = Sequential()
model_LSTM.add(Embedding(vocabulary_size, 128, input_length=maxlen))
model_LSTM.add(LSTM(64,activation='tanh'))
model_LSTM.add(Dense(1,activation='sigmoid'))
model_LSTM.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 1853, 128)         6820992   
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 6,870,465
Trainable params: 6,870,465
Non-trainable params: 0
_________________________________________________________________


In [24]:
model_LSTM.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics = 'accuracy')

In [25]:
model_LSTM.fit(padded_train_sequences,y_train, validation_split=.2, epochs= 5, batch_size=256)

Epoch 1/5
25/25 [==============================] - 543s 22s/step - loss: 0.6384 - accuracy: 0.6363 - val_loss: 0.8398 - val_accuracy: 0.5300
Epoch 2/5
25/25 [==============================] - 728s 29s/step - loss: 0.4565 - accuracy: 0.8152 - val_loss: 0.4267 - val_accuracy: 0.8263
Epoch 3/5
25/25 [==============================] - 1016s 41s/step - loss: 0.3240 - accuracy: 0.8744 - val_loss: 0.3674 - val_accuracy: 0.8444
Epoch 4/5
25/25 [==============================] - 1202s 48s/step - loss: 0.2637 - accuracy: 0.9017 - val_loss: 0.3297 - val_accuracy: 0.8575
Epoch 5/5
25/25 [==============================] - 1240s 50s/step - loss: 0.1972 - accuracy: 0.9308 - val_loss: 0.3611 - val_accuracy: 0.8444


In [27]:
model_LSTM.evaluate(padded_test_sequences, y_test)

63/63 [==============================] - 33s 529ms/step - loss: 0.3699 - accuracy: 0.8405


[0.3699425458908081, 0.840499997138977]

**GRU Model**

In [28]:
from keras.models import Sequential
from keras.layers import Embedding, GRU, Dense

model_GRU = Sequential()
model_GRU.add(Embedding(vocabulary_size, 128, input_length=maxlen))
model_GRU.add(GRU(64,activation='tanh'))
model_GRU.add(Dense(1,activation='sigmoid'))
model_GRU.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 1853, 128)         6820992   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 6,858,305
Trainable params: 6,858,305
Non-trainable params: 0
_________________________________________________________________


In [29]:
model_GRU.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = 'accuracy')

In [30]:
model_GRU.fit(padded_train_sequences, y_train, validation_split = .2, epochs = 5, batch_size=256)

Epoch 1/5
25/25 [==============================] - 1153s 46s/step - loss: 0.6798 - accuracy: 0.6161 - val_loss: 0.5907 - val_accuracy: 0.7119
Epoch 2/5
25/25 [==============================] - 1212s 49s/step - loss: 0.4723 - accuracy: 0.8053 - val_loss: 0.5350 - val_accuracy: 0.7262
Epoch 3/5
25/25 [==============================] - 1142s 45s/step - loss: 0.3335 - accuracy: 0.8636 - val_loss: 0.4039 - val_accuracy: 0.8206
Epoch 4/5
25/25 [==============================] - 1016s 41s/step - loss: 0.2411 - accuracy: 0.9042 - val_loss: 0.4086 - val_accuracy: 0.8169
Epoch 5/5
25/25 [==============================] - 1014s 41s/step - loss: 0.1950 - accuracy: 0.9244 - val_loss: 0.5434 - val_accuracy: 0.7650


In [32]:
model_GRU.evaluate(padded_test_sequences, y_test)

63/63 [==============================] - 22s 354ms/step - loss: 0.4900 - accuracy: 0.7880


[0.49001914262771606, 0.7879999876022339]


**Bidirectional LSTM Model**



In [33]:
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional,  LSTM, Dense

model_BiLSTM = Sequential()
model_BiLSTM.add(Embedding(vocabulary_size, 128, input_length=maxlen))
model_BiLSTM.add(Bidirectional(LSTM(64,activation='tanh')))
model_BiLSTM.add(Dense(1,activation='sigmoid'))
model_BiLSTM.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 1853, 128)         6820992   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 6,919,937
Trainable params: 6,919,937
Non-trainable params: 0
_________________________________________________________________
